In [457]:
#import datetime as dt
#from datetime import date
#import os
#import sys
import numpy as np
import pandas as pd
#from glob import glob
#import sqlalchemy
#import pymysql
#import time, datetime
#from datetime import datetime, timedelta
#from datetime import timedelta, date
#import time
from workalendar.america import BrazilBankCalendar
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
#pd.set_option('display.max_columns', 500)
#import warnings
#warnings.filterwarnings("ignore")
from scipy.stats import kurtosis
import statsmodels.stats.api as sms
from statsmodels.compat import lzip
from statsmodels.graphics.gofplots import qqplot
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.stats.diagnostic import het_breuschpagan, het_goldfeldquandt,het_white
from statsmodels.stats.diagnostic import linear_harvey_collier, linear_reset, spec_white
from statsmodels.stats.diagnostic import linear_rainbow
from statsmodels.graphics.regressionplots import plot_leverage_resid2
from statsmodels.stats.outliers_influence import OLSInfluence, variance_inflation_factor


In [473]:
###Importando as bases
##Criando colunas:
#detrator / neutro / promotor / [FLAG] Entrega x nps
from numpy import unicode_

base = pd.read_csv(r'../NPS_Bulky/analise_dummy_ou_outros (version 1).csv',sep=';', encoding='ISO-8859-1')
base.shape


#Cria as colunas de soma das dummys
def somase(data_frame):

    colunas = ['Flag Atraso Transporte2','Flag Atraso Fornecedor3','Flag Atraso Coleta4','Flag Atraso Cliente5','[FLAG]Cancelado',
    '[FLAG]Recompra_ff','[FLAG]Erro prazo transporte validacao','[FLAG]Cross','[FLAG]Atraso_criacao_OC','[FLAG]Reprogramacao_OC',
    '[FLAG]Atraso Fornecedor','[FLAG] Entregue','[FLAG] Pratica_indevida','[FLAG] Sac_pos_entrega','[FLAG] Reversa','[FLAG] PJ',
    '[FLAG] Mais de uma entrega','[FLAG]Importacao WMS','[FLAG] Recompra','[FLAG] Entrega_Mista','[FLAG] Virada_Faturamento',
    'FLAG - Infos_encontradas','[FLAG] Atraso limite','[FLAG] Oc sem bip','[FLAG] Insucesso','[FLAG] End. N.Localizado/Ausente',
    '[FLAG] Tempo Habil','[FLAG] Atraso Redespacho','[FLAG] Diarias','[FLAG] Atraso Coletas','[FLAG] Leves','[FLAG] Atraso CD',
    '[FLAG] Recompra assistencia','[FLAG] Recompra Total','[FLAG] Origem Recompra','[FLAG] Atraso Cliente*','FLAG_Parceiro',
    'FLAG_terceiras_parceiras','[FLAG] - Detrator_mais_entrega','[FLAG] - Categorias_detrator','[FLAG] Private_Label','Guide_flag'
                ]
   
    df_retorno = data_frame
    for coluna in colunas:
        try:
            nova_coluna = df_retorno.groupby('pv_pai')[coluna].sum()
            df_retorno = df_retorno.merge(nova_coluna, left_on='pv_pai', right_on='pv_pai',how='left')
            df_retorno.rename(columns={coluna + '_y': coluna + '_soma'}, inplace=True,errors='raise')
            # print('realizado colunas: ', coluna + '_soma')
        except Exception as e:
            print('deu ruim: ', e)
    return df_retorno

#funcao dias
def dia_util(d1, d2):
    cal = BrazilBankCalendar()
    cal.include_ash_wednesday = False
    a = d1
    b = d2
    if pd.isnull(d1) or pd.isnull(d2):
        return np.nan
    else:
        a = pd.to_datetime(d1, infer_datetime_format=True, dayfirst=True)
        b = pd.to_datetime(d2, infer_datetime_format=True, dayfirst=True)
        if b.weekday() in [5, 6]:
            if b > a:
                resp = (cal.get_working_days_delta(a, b)) + 1
            else:
                resp = (cal.get_working_days_delta(a, b))
                resp = resp * -1
        else:
            resp = (cal.get_working_days_delta(a, b))
            if a > b:
                return -resp
    return resp

#regra flag aging x nps:
def flag_nps_x_entrega(limite_cliente, data_entregue,qtd_dias_atraso):
    if dia_util(limite_cliente, data_entregue) > qtd_dias_atraso:
        return int(1)
    else:
        return int(0)

    
#Roda a função somase com a base importada
base = somase(base)

#Cria as colunas de detrator, neutro e promotor
def dummy_detra(resposta):
    if resposta <= 6:
        return "1"
    else:
        return "0"

def neutro(resposta):
    if resposta >= 7 and resposta <= 8:
        return "1"
    else:
        return "0"

def promotor (resposta):
    if resposta >= 9:
        return '1'
    else:
        return '0'

#criacao de colunas:
base['detrator'] = base.apply(
        lambda row: dummy_detra(row['customer_answer']), axis=1)

base['neutro'] = base.apply(
lambda row: neutro(row['customer_answer']), axis=1)

base['promotor'] = base.apply(
        lambda row: promotor(row['customer_answer']), axis=1)

#####flag entrega x nps, mudar na funcao o numero de dias:
base['[FLAG] Entrega x nps'] = base.apply(
        lambda row: flag_nps_x_entrega(row['data_limite_cliente'], row['data_entregue'], 3), axis=1)

#Drop colunas desnecessárias
base = base.drop(columns=['3. Sac_pos_entrega_s','97 Atraso WMS_s','96 Recompra_s','Reclamacao_cliente_s','Info_Encontradas_s','Coluna34','Coluna4',
'Coluna2','cidade_cliente','pedped','cep','chave_nfm','chave_ped','nf_fornecedor','fornecedor','nf_mi','data_compra_cliente','sku',
'categoria_old','categoria_new','head_categoria','descricao','chave_pedped','pedido_x','origem','destino','tp_t3','data_criacao_oc',
'data_aprovacao','data_limite_fornecedor','data_expedicao','data_limite_cliente','data_limite_coleta','data_limite_transporte','data_bip',
'data_entregue','pz_total_x','nota_nfd','tp_nf','tp_eagle','status_pedido_portal','Atraso Local V1','OC','Motivo_Recompra','prazo_interno_x',
'prazo_interno_y','channel','Virada_FF_VAO','Pesso Juridica','pedido_prazo_fornecedor','pedido_prazo_transportadora','cotacao_prazo_transporte',
'cotacao_prazo_fornecedor','Título Romaneio','Motorista','Roteiro','Prioridade','Codigo_EDI','EDI','Importacao_FF','Atraso Local V2',
'92.Virada_ff_Vao','91.Erro de Prazo Transporte','89.Reversa','1. Depara Transportadora','2. Pratica_indevida','3. Sac_pos_entrega',
'97 Atraso WMS','96 Recompra','Consolidador','Data_resposta','Media_dia_x_resposta','Motivo_sac Entrega','Reclamacao_cliente','chave_TP',
'DePara_TP_Parceiro:','Rank_Regras','[Prazo] aux','Dia Sem. Aprovação','Feriado','Prazo Vendido','Delta prazo cotado',
'Data Limite Cotado','aux Comparação Prazo Total (Cotado x Vendido)','Aux_coleta','Prazo_interno','Prazo_x_exp','CEP2','Pedido_original_Recompra',
'Data_Limite_para_Recompra','nova_data_limite','Aging_cliente','data-entrega','Chave_espaco_2','Fornecedor_ocr_desc',
'Guide','Categoria_nova_estrutura','Cabe_casa',
'Yes_casa','% detrator_categorias','qtd. categoria_detrator','1 cat','2 cat','3 cat','4 cat','5 cat','Loja',
'[compra] Foi fácil realizar sua compra?','Estado_destino_sigla','Semana','Mês',
'[pagamento] Como foi a sua experiência com relação ao pagamento do produto adquirido?','Como foi sua experiência enquanto esperava o seu produto?','[entrega] Você ficou satisfeito(a) com a entrega?',
'[produto] O produto atendeu às suas expectativas?','[atendimento] Você ficou satisfeito com nosso atendimento?','[entrega]] A embalagem chegou com problemas (rasgada, danificada, molhada...)',
'[entrega]] A entrega foi feita fora do prazo','[[entrega]] Achei o prazo de entrega muito longo.]',
'[[entrega]] Achei o valor do frete muito alto.]','[[entrega]] Não foi possível agendar a entrega.]',
'[atendimento]] (aberta) Outro. Por favor, especifique:]','(aberta) Como foi sua experiência enquanto esperava o seu produto?',
'[compra]] (aberta) Foi fácil realizar sua compra?]','[entrega]] (aberta) Por favor, nos conte mais sobre a sua experiência.',
'[entrega]] (aberta) Você ficou satisfeito(a) com a entrega?','[pagamento]] (aberta) Visando melhorar essa experiência, você pode contar quais d',
'[produto]] (aberta) Por favor, nos conte mais sobre a sua resposta','Atraso_Transp_s','Flag Atraso Fornecedor_s','Flag Atraso Coleta_s','Flag Atraso Cliente_s',
'[FLAG]Cancelado_S','[FLAG]Recompra_ff_S','[FLAG]Erro prazo transporte validacao_S','96.Atraso_importação_WMS_S',
'[FLAG]Cross_S','[FLAG]Atraso_criacao_OC_S','[FLAG]Reprogramacao_OC_S','92.Virada_ff_Vao_S','91.Erro de Prazo Transporte_S',
'[FLAG]Atraso Fornecedor_S','89.Reversa_S','NPS_NOTA_S','[FLAG] Entregue_S','[FLAG] Pratica_indevida_s','[FLAG] Sac_pos_entrega_s',
'[FLAG] Reversa_s','[FLAG] PJ_s','[FLAG] Mais de uma entrega_s','[FLAG]Importacao WMS_s','[FLAG] Recompra_s',
'[FLAG] Entrega_Mista_s','[FLAG] Virada_Faturamento_s','FLAG - Infos_encontradas_s','[FLAG] Atraso limite_s',
'[FLAG] Oc sem bip_s','[FLAG] Insucesso_s','[FLAG] End. N.Localizado/Ausente_s','[FLAG] Tempo Habil_s',
'[FLAG] Atraso Redespacho_s','[FLAG] Diarias_s','Flag Atraso Transporte2_x','Flag Atraso Fornecedor3_x',
'Flag Atraso Coleta4_x','Flag Atraso Cliente5_x','[FLAG]Cancelado_x','[FLAG]Recompra_ff_x','[FLAG]Erro prazo transporte validacao_x',
'96.Atraso_importação_WMS','[FLAG]Cross_x','[FLAG]Atraso_criacao_OC_x','[FLAG]Reprogramacao_OC_x','[FLAG]Atraso Fornecedor_x','NPS_NOTA',
'[FLAG] Entregue_x','[FLAG] Pratica_indevida_x','[FLAG] Sac_pos_entrega_x','[FLAG] Reversa_x','[FLAG] PJ_x','[FLAG] Mais de uma entrega_x',
'[FLAG]Importacao WMS_x','[FLAG] Recompra_x','[FLAG] Entrega_Mista_x','[FLAG] Virada_Faturamento_x','Info_Encontradas',
'FLAG - Infos_encontradas_x','[FLAG] Atraso limite_x','[FLAG] Oc sem bip_x','[FLAG] Insucesso_x',
'[FLAG] End. N.Localizado/Ausente_x','[FLAG] Tempo Habil_x','[FLAG] Atraso Redespacho_x',
'[FLAG] Diarias_x','[FLAG] Atraso Coletas_x','[FLAG] Leves_x','[FLAG] Atraso CD_x','[FLAG] Recompra assistencia_x',
'[FLAG] Recompra Total_x','[FLAG] Origem Recompra_x','[FLAG] Atraso Cliente*_x','FLAG_Parceiro_x','FLAG_terceiras_parceiras_x',
'[FLAG] - Detrator_mais_entrega_x','[FLAG] - Categorias_detrator_x','[FLAG] Private_Label_x','Guide_flag_x',
'[FLAG] Origem Recompra_soma',
'FLAG_terceiras_parceiras_soma',
'tipo_venda','Guide_flag_soma','[FLAG] Entregue_soma', 'created_at','tipo_pedido_nps','[FLAG]Recompra_ff_soma','[FLAG] PJ_soma','FLAG - Infos_encontradas_soma','[FLAG] Recompra assistencia_soma','[FLAG] Recompra Total_soma','[FLAG] Atraso Cliente*_soma','FLAG_Parceiro_soma','[FLAG] - Detrator_mais_entrega_soma','[FLAG] - Categorias_detrator_soma',
])



In [474]:
#Transforma em pedido (cada linha um pedido pai)
base = base.drop_duplicates(subset='pv_pai', keep='first')

base.head()


,customer_answer,pv_pai,Flag Atraso Transporte2_soma,Flag Atraso Fornecedor3_soma,Flag Atraso Coleta4_soma,Flag Atraso Cliente5_soma,[FLAG]Cancelado_soma,[FLAG]Erro prazo transporte validacao_soma,[FLAG]Cross_soma,[FLAG]Atraso_criacao_OC_soma,[FLAG]Reprogramacao_OC_soma,[FLAG]Atraso Fornecedor_soma,[FLAG] Pratica_indevida_soma,[FLAG] Sac_pos_entrega_soma,[FLAG] Reversa_soma,[FLAG] Mais de uma entrega_soma,[FLAG]Importacao WMS_soma,[FLAG] Recompra_soma,[FLAG] Entrega_Mista_soma,[FLAG] Virada_Faturamento_soma,[FLAG] Atraso limite_soma,[FLAG] Oc sem bip_soma,[FLAG] Insucesso_soma,[FLAG] End. N.Localizado/Ausente_soma,[FLAG] Tempo Habil_soma,[FLAG] Atraso Redespacho_soma,[FLAG] Diarias_soma,[FLAG] Atraso Coletas_soma,[FLAG] Leves_soma,[FLAG] Atraso CD_soma,[FLAG] Private_Label_soma,detrator,neutro,promotor,[FLAG] entrega x nps
0,10,28162012,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,2,0.0,0,2.0,0,0.0,0.0,0,0,0,0.0,0.0,0,0.0,0,0,0,0,1,0
1,10,28096157,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0,0.0,0,0.0,0,0.0,0.0,0,0,0,0.0,0.0,0,0.0,0,0,0,0,1,0
2,10,27804324,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0,0.0,0,0.0,0,0.0,0.0,0,0,0,0.0,0.0,0,0.0,0,0,0,0,1,0
3,10,28032244,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0,0.0,0,0.0,0,0.0,0.0,0,0,0,0.0,0.0,0,0.0,0,1,0,0,1,0
4,10,28214644,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0,0.0,0,0.0,0,0.0,0.0,0,0,0,0.0,0.0,0,0.0,0,0,0,0,1,0


In [475]:
###drop mais colunas q eu n quero
X_vars = base.drop(columns=['[FLAG]Cancelado_soma','[FLAG] Leves_soma','[FLAG] Mais de uma entrega_soma','[FLAG] End. N.Localizado/Ausente_soma','[FLAG] Tempo Habil_soma','pv_pai',
'Flag Atraso Transporte2_soma','Flag Atraso Fornecedor3_soma','Flag Atraso Coleta4_soma', '[FLAG]Erro prazo transporte validacao_soma','[FLAG]Cross_soma', '[FLAG]Atraso_criacao_OC_soma', 
'[FLAG]Reprogramacao_OC_soma', '[FLAG]Atraso Fornecedor_soma', '[FLAG]Importacao WMS_soma', '[FLAG] Entrega_Mista_soma','[FLAG] Virada_Faturamento_soma', '[FLAG] Atraso limite_soma', 
'[FLAG] Oc sem bip_soma', '[FLAG] Atraso Redespacho_soma', '[FLAG] Diarias_soma','[FLAG] Atraso Coletas_soma', '[FLAG] Atraso CD_soma', 'detrator','neutro','promotor'])


In [476]:
###Criando colunas multiplicadas, e transformando formato para números

##for para realizar as colunas mutiplicadas
colunas = ['Flag Atraso Cliente5_soma', '[FLAG] Pratica_indevida_soma','[FLAG] Sac_pos_entrega_soma','[FLAG] Reversa_soma', '[FLAG] Recompra_soma', '[FLAG] Insucesso_soma', '[FLAG] Private_Label_soma']  

##funcao nao sendo utilizada, faz as colunas seram ^2
def coluna_ao_quadrado():
    for j in range(X_vars.shape[1]):
        try:
            for coluna in colunas:
                c = 0
                if coluna != colunas[j]:
                    nova_coluna = X_vars[colunas[j] + "_x_"+ coluna] = X_vars[colunas[j]] ^ 2
                    c = c +1 
        except:
            print('finalizado') 


##mutiplicando colunas:
for j in range(X_vars.shape[1]):
    try:
        for coluna in colunas:
            c = 0
            if coluna != colunas[j]:
                nova_coluna = X_vars[colunas[j] + "_x_"+ coluna] = X_vars[colunas[j]] * X_vars[colunas[c]]
                c = c +1 
    except:
        print('finalizado') 


##transformando colunas em inteiros
col = X_vars.columns.values.tolist()
col
X_vars.where(X_vars == '', 0)
print(len(col))
for coluna in col:
    try: 
        X_vars[coluna] = X_vars[coluna].astype(str)
        X_vars[coluna] = X_vars[coluna].str.replace('.0', '')
        X_vars[coluna] = X_vars[coluna].astype(int)
        # print(X_vars[coluna].dtypes)
    except:
        try:
            X_vars[coluna] = X_vars[coluna].astype(str)
            X_vars[coluna] = X_vars[coluna].str.replace('', '0')
            X_vars[coluna] = X_vars[coluna].astype(int)
        except Exception as e:
            print('Deu erro na formatação: ',e)
            


#print data_frame:
X_vars.head()


finalizado
finalizado
51


,customer_answer,Flag Atraso Cliente5_soma,[FLAG] Pratica_indevida_soma,[FLAG] Sac_pos_entrega_soma,[FLAG] Reversa_soma,[FLAG] Recompra_soma,[FLAG] Insucesso_soma,[FLAG] Private_Label_soma,[FLAG] entrega x nps,Flag Atraso Cliente5_soma_x_[FLAG] Pratica_indevida_soma,Flag Atraso Cliente5_soma_x_[FLAG] Sac_pos_entrega_soma,Flag Atraso Cliente5_soma_x_[FLAG] Reversa_soma,Flag Atraso Cliente5_soma_x_[FLAG] Recompra_soma,Flag Atraso Cliente5_soma_x_[FLAG] Insucesso_soma,Flag Atraso Cliente5_soma_x_[FLAG] Private_Label_soma,[FLAG] Pratica_indevida_soma_x_Flag Atraso Cliente5_soma,[FLAG] Pratica_indevida_soma_x_[FLAG] Sac_pos_entrega_soma,[FLAG] Pratica_indevida_soma_x_[FLAG] Reversa_soma,[FLAG] Pratica_indevida_soma_x_[FLAG] Recompra_soma,[FLAG] Pratica_indevida_soma_x_[FLAG] Insucesso_soma,[FLAG] Pratica_indevida_soma_x_[FLAG] Private_Label_soma,[FLAG] Sac_pos_entrega_soma_x_Flag Atraso Cliente5_soma,[FLAG] Sac_pos_entrega_soma_x_[FLAG] Pratica_indevida_soma,[FLAG] Sac_pos_entrega_soma_x_[FLAG] Reversa_soma,[FLAG] Sac_pos_entrega_soma_x_[FLAG] Recompra_soma,[FLAG] Sac_pos_entrega_soma_x_[FLAG] Insucesso_soma,[FLAG] Sac_pos_entrega_soma_x_[FLAG] Private_Label_soma,[FLAG] Reversa_soma_x_Flag Atraso Cliente5_soma,[FLAG] Reversa_soma_x_[FLAG] Pratica_indevida_soma,[FLAG] Reversa_soma_x_[FLAG] Sac_pos_entrega_soma,[FLAG] Reversa_soma_x_[FLAG] Recompra_soma,[FLAG] Reversa_soma_x_[FLAG] Insucesso_soma,[FLAG] Reversa_soma_x_[FLAG] Private_Label_soma,[FLAG] Recompra_soma_x_Flag Atraso Cliente5_soma,[FLAG] Recompra_soma_x_[FLAG] Pratica_indevida_soma,[FLAG] Recompra_soma_x_[FLAG] Sac_pos_entrega_soma,[FLAG] Recompra_soma_x_[FLAG] Reversa_soma,[FLAG] Recompra_soma_x_[FLAG] Insucesso_soma,[FLAG] Recompra_soma_x_[FLAG] Private_Label_soma,[FLAG] Insucesso_soma_x_Flag Atraso Cliente5_soma,[FLAG] Insucesso_soma_x_[FLAG] Pratica_indevida_soma,[FLAG] Insucesso_soma_x_[FLAG] Sac_pos_entrega_soma,[FLAG] Insucesso_soma_x_[FLAG] Reversa_soma,[FLAG] Insucesso_soma_x_[FLAG] Recompra_soma,[FLAG] Insucesso_soma_x_[FLAG] Private_Label_soma,[FLAG] Private_Label_soma_x_Flag Atraso Cliente5_soma,[FLAG] Private_Label_soma_x_[FLAG] Pratica_indevida_soma,[FLAG] Private_Label_soma_x_[FLAG] Sac_pos_entrega_soma,[FLAG] Private_Label_soma_x_[FLAG] Reversa_soma,[FLAG] Private_Label_soma_x_[FLAG] Recompra_soma,[FLAG] Private_Label_soma_x_[FLAG] Insucesso_soma
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [477]:
X_vars = X_vars.drop(columns=['customer_answer'])
regressao = sm.OLS(base['customer_answer'],X_vars).fit()
print(regressao.summary())
# X_vars.to_csv(r'C:\Users\lucas.valdomiro\Documents\GitHub\NPS_Bulky\base_multiplica.csv', sep=';')

                                 OLS Regression Results                                
Dep. Variable:        customer_answer   R-squared (uncentered):                   0.243
Model:                            OLS   Adj. R-squared (uncentered):              0.243
Method:                 Least Squares   F-statistic:                              404.1
Date:                Mon, 07 Nov 2022   Prob (F-statistic):                        0.00
Time:                        07:23:15   Log-Likelihood:                         -65431.
No. Observations:               18858   AIC:                                  1.309e+05
Df Residuals:                   18843   BIC:                                  1.310e+05
Df Model:                          15                                                  
Covariance Type:            nonrobust                                                  
                                                                 coef    std err          t      P>|t|      [0.025      

In [471]:



selecionado = base.drop(columns=['[FLAG] - Categorias_detrator_soma','[FLAG] - Detrator_mais_entrega_soma','Flag Atraso Cliente5_soma','FLAG_Parceiro_soma','pv_pai','[FLAG]Cancelado_soma','[FLAG]Recompra_ff_soma','[FLAG] PJ_soma','[FLAG] Recompra assistencia_soma','[FLAG] Recompra Total_soma','[FLAG] Atraso Cliente*_soma','FLAG_Parceiro_soma','[FLAG] Private_Label_soma'])
# detrator = detrator.drop(columns=['pv_pai','[FLAG]Cancelado_soma','[FLAG]Recompra_ff_soma','[FLAG] PJ_soma','FLAG - Infos_encontradas_soma','[FLAG] Recompra assistencia_soma','[FLAG] Recompra Total_soma','[FLAG] Atraso Cliente*_soma','FLAG_Parceiro_soma','[FLAG] - Detrator_mais_entrega_soma','[FLAG] - Categorias_detrator_soma','[FLAG] Private_Label_soma'])
# neutro =  neutro.drop(columns=['pv_pai','[FLAG]Cancelado_soma','[FLAG]Recompra_ff_soma','[FLAG] PJ_soma','FLAG - Infos_encontradas_soma','[FLAG] Recompra assistencia_soma','[FLAG] Recompra Total_soma','[FLAG] Atraso Cliente*_soma','FLAG_Parceiro_soma','[FLAG] - Detrator_mais_entrega_soma','[FLAG] - Categorias_detrator_soma','[FLAG] Private_Label_soma'])
# promotor =  promotor.drop(columns=['pv_pai','[FLAG]Cancelado_soma','[FLAG]Recompra_ff_soma','[FLAG] PJ_soma','FLAG - Infos_encontradas_soma','[FLAG] Recompra assistencia_soma','[FLAG] Recompra Total_soma','[FLAG] Atraso Cliente*_soma','FLAG_Parceiro_soma','[FLAG] - Detrator_mais_entrega_soma','[FLAG] - Categorias_detrator_soma','[FLAG] Private_Label_soma'])



###detrator:
plt.figure(figsize=(35,8))
sns.heatmap(selecionado.corr(), annot = True, cmap= "RdYlGn", color='blue');
plt.title('Correlação de Pearson',size=10);


KeyError: "['[FLAG] - Categorias_detrator_soma', '[FLAG] - Detrator_mais_entrega_soma', 'FLAG_Parceiro_soma', '[FLAG]Recompra_ff_soma', '[FLAG] PJ_soma', '[FLAG] Recompra assistencia_soma', '[FLAG] Recompra Total_soma', '[FLAG] Atraso Cliente*_soma', 'FLAG_Parceiro_soma'] not found in axis"

In [16]:
#coeficiente de determinacao r²
modelo.score(x,y)

1.0

In [17]:
#Intercepto (ou constante) do modelo.
modelo.intercept_

7.232998910922106e-08

In [18]:
# Coeficientes (ou parâmetros) do modelo.
modelo.coef_

array([ 1.00000000e+00, -2.61863804e-15, -1.43567063e-14, -6.37754190e-15,
       -3.72951105e-15, -3.13107703e-14,  8.16049524e-14,  5.90856407e-16,
        1.11808602e-14, -3.59025660e-14, -5.36210192e-15,  5.14037608e-16,
       -5.31536269e-15,  7.10399129e-14,  7.14860376e-15,  1.70636724e-14,
        3.33263409e-15,  1.73083650e-14,  3.10509213e-15, -2.17407967e-14,
       -8.42947591e-15, -3.04315952e-14,  2.73294734e-14,  4.24929188e-16,
        2.25364806e-16,  1.99564382e-16, -2.95889852e-14,  4.68679401e-14,
       -3.72951105e-15, -2.09383773e-14, -5.33716735e-14, -2.08119022e-14,
        4.46901721e-14, -1.85566104e-14,  2.27985292e-14, -3.69143475e-15,
        2.52667835e-15, -4.19916855e-15,  3.50190295e-15, -2.41530785e-15])

In [191]:
# Gerando os modelos com a Pingouin
# Irei gerar dois modelos: um com intercepto e outro sem o intercepto. Porém, antes, terei que transformar as variáveis para o formato numpy.

Xp = x.to_numpy()
yp = y.to_numpy()

In [78]:

from numpy import float64


r_q = pd.read_csv(r"D:\r_quadrado.csv",sep=';', encoding='ISO-8859-1')

drop = r_q.drop(columns=['e'])


def transforma_col(df_base):
    colunas = ['[FLAG] Sac_pos_entrega_soma','[FLAG] Reversa_soma','[FLAG] Mais de uma entrega_soma','[FLAG] Recompra_soma','[FLAG] - Categorias_detrator_soma']
    for coluna in colunas:
        try:
            print('Modificando colunas para número: ',coluna)
            df_base[coluna] = df_base[coluna].fillna(0).astype(int)
        except Exception as e:
            print('verificar, erro format: ', e)
     
drop.dtypes


sm.OLS(drop['e'], drop.drop(columns=['e'])).fit()
# plt.figure(figsize=(5,6))
# sns.heatmap(drop.corr(), annot = True, cmap= "RdYlGn");
# plt.title('Correlação de Pearson',size=10);


KeyError: 'e'